In [ ]:
import json
import numpy
import os
import shutil
import zipfile
from google.colab import drive
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Разархивирование книг и историй сообщений из чатов телеграм

In [ ]:
books_path = "/content/drive/MyDrive/CycleGAN_for_TST_problem/books_txt.zip"
with zipfile.ZipFile(books_path, "r") as zip_ref:
    for filename in zip_ref.namelist():
        zip_ref.extract(filename, "/content/books")

In [ ]:
tg_messages_path = "/content/drive/MyDrive/CycleGAN_for_TST_problem/telegram_comments.zip"
with zipfile.ZipFile(tg_messages_path, "r") as zip_ref:
    for filename in zip_ref.namelist():
        if "telegram/telegram/" not in filename:
            zip_ref.extract(filename, "/content/tg")

## Парсинг предложений из книг, небольшая предобработка и создание Pandas Series

In [ ]:
low_chars_border = 8
high_chars_border = 200

In [ ]:
lit_data = set()
all_files = os.listdir("/content/books/books_txt/")
for file in all_files:
    file_txt = open(f"/content/books/books_txt/{file}")
    data_txt = file_txt.read()
    sentences = sent_tokenize(data_txt)
    for sentence in sentences:
        sentence = sentence.strip()
        sentence = re.sub("[^а-яА-Я–-—,.!?:«» ]", "", sentence)
        start_removed = 0
        while len(sentence) > 0 and sentence[0] in "–-—,.!?:«» ":
            start_removed += 1
        sentence = sentence[start_removed:]
        if low_chars_border < len(sentence) < high_chars_border:
            lit_data.add(sentence)

In [ ]:
lit = pd.Series(list(lit_data))
lit

,0
0,Пьер с Анной Михайловной прошли в .
1,На пороге дома барон остановился в нерешительн...
2,Вся церемония продолжалась около часа.
3,"Вы, видите ли, еще, может, надумали бы дать ей..."
4,Во избежание такой неприятности я приостановил...
...,...
1382544,А без Ларсона Макмерфи ничего не стоит.
1382545,"Он приедет, сэр."
1382546,"Я вдруг почувствовал, что под холодной водой т..."
1382547,"Фабрицио, со своей стороны, не мог простить се..."


## Парсинг предложений из сообщений Telegram, небольшая предобработка и создание Pandas Series

In [ ]:
comnts = set()
allFiles = os.listdir("/content/tg/telegram/")
for filename in allFiles:
    if len(comnts) > len(lit) - 1:
                break
    with open(f"/content/tg/telegram/{filename}", 'r') as f:
            groupedComments = json.load(f)
            for comment in groupedComments:
                row = comment["text"]
                sentences = sent_tokenize(row)
                sentences = sent_tokenize(data_txt)
                for sentence in sentences:
                    sentence = sentence.strip()
                    sentence = re.sub("[^а-яА-Я–-—,.!?:«» ]", "", sentence)
                    start_removed = 0
                    while len(sentence) > 0 and sentence[0] in "–-—,.!?:«» ":
                        start_removed += 1
                    sentence = sentence[start_removed:]
                    if low_chars_border < len(sentence) < high_chars_border:
                        comnts.add(sentence)
                        if len(comnts) > len(lit) - 1:
                            break
                if len(comnts) > len(lit) - 1:
                            break

In [ ]:
com = pd.Series(list(comnts))
com

,0
0,За это ему платили по тысяч за раз.
1,больше почти ничего не помню
2,"ребята,срочняк косарь нужен подкинете на постр..."
3,некоторые из любопытства заходят на твою стран...
4,Пойду дальше рен ТВ смотреть
...,...
1382544,Бульон какойнибудь хотя бы
1382545,Будешь бабойрыбой
1382546,где будет
1382547,"Леш, теперь можно покупать"


## Объединение Pandas Series в один Pandas Dataframe, сохранение его на диск

In [ ]:
data = pd.DataFrame({"lit_text": lit, "tg_text": com})
data

,lit_text,tg_text
0,Пьер с Анной Михайловной прошли в .,За это ему платили по тысяч за раз.
1,На пороге дома барон остановился в нерешительн...,больше почти ничего не помню
2,Вся церемония продолжалась около часа.,"ребята,срочняк косарь нужен подкинете на постр..."
3,"Вы, видите ли, еще, может, надумали бы дать ей...",некоторые из любопытства заходят на твою стран...
4,Во избежание такой неприятности я приостановил...,Пойду дальше рен ТВ смотреть
...,...,...
1382544,А без Ларсона Макмерфи ничего не стоит.,Бульон какойнибудь хотя бы
1382545,"Он приедет, сэр.",Будешь бабойрыбой
1382546,"Я вдруг почувствовал, что под холодной водой т...",где будет
1382547,"Фабрицио, со своей стороны, не мог простить се...","Леш, теперь можно покупать"


In [ ]:
data.to_csv("/content/drive/MyDrive/CycleGAN_for_TST_problem/data.csv", index=False)